In [49]:
import requests
import time
from IPython.display import JSON
import pandas as pd
import configparser as cp
import sqlalchemy as db
from datetime import datetime

In [41]:
"""
    Reads the config file and returns the config object
"""
config = cp.ConfigParser()
config.read('config.ini')

['config.ini']

In [42]:
def connect_to_db() -> db.engine.base.Connection:
    """
        Connect to a postgres database using an url from config.ini file

        Returns:
            connection: sqlalchemy connection object
    """
    engine = db.create_engine(config['DB']['URL'])
    connection = engine.connect()
    return connection

In [43]:
rate_limit = requests.get(
    config['API']['URL'] +
    config['API']['RATE_LIMIT'] ,
    auth=(
        config['API']['USER'],    
        config['API']['KEY']
    )   
).json()["resources"]["search"] 

print (f"Seconds till reset: { rate_limit['reset'] - int(time.time()) }")
print (f"Seconds between requests: { (rate_limit['reset'] - int(time.time()) )/ rate_limit['remaining']  }")


Seconds till reset: 61
Seconds between requests: 2.033333333333333


In [54]:
JSON(
    requests.get( 
        config['API']['URL'] +
        config['API']['REPOS'],
        auth=(
            config['API']['USER'],
            config['API']['KEY']
        )
    ).json()["items"][0]
)

<IPython.core.display.JSON object>